In [1]:
import numpy as np
import sklearn as sk
from tensorflow import keras
import pandas as pd
import scipy
import tensorflow as tf

import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import tensorflow_docs as tfdocs
np.set_printoptions(suppress=True)
import tensorflow_docs.plots
import tensorflow_docs.modeling
from tensorflow.keras import regularizers, layers

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
%matplotlib inline
pd.set_option('display.max_columns', 150)

In [ ]:
data_un = pd.read_csv('train_data.csv', sep = ',')

In [ ]:
    def _preprocessor(self, X_raw, training = False):
        """

        This function prepares the features of the data for training,
        evaluation, and prediction.

        Parameters
        ----------
        X_raw : Pandas dataframe
            This is the raw data features excluding claims information 

        Returns
        -------
        X: Pandas DataFrame
            A clean data set that is used for training and prediction.
        """
        # =============================================================
        # YOUR CODE HERE

        columns_x = ['pol_bonus', 'pol_coverage', 'pol_duration',
                     'pol_sit_duration', 'pol_pay_freq', 'pol_payd', 'pol_usage',
                     'drv_drv2', 'drv_age1', 'drv_age2', 'drv_sex1',
                     'drv_sex2', 'drv_age_lic1', 'drv_age_lic2', 'vh_age', 'vh_cyl',
                     'vh_din', 'vh_fuel', 'vh_make', 'vh_sale_begin',
                     'vh_sale_end', 'vh_speed', 'vh_type', 'vh_value', 'vh_weight',
                     'town_mean_altitude', 'town_surface_area', 'population', 'city_district_code',
                     ]

        x_data_un = X_raw[columns_x]

        cat_to_int_dict = {'pol_coverage': {'Mini': 0, 'Median1': 1, 'Median2': 2, 'Maxi': 3},
                           'pol_pay_freq': {'Monthly': 0, 'Quarterly': 1, 'Biannual': 2, 'Yearly': 3},
                           'pol_payd': {'No': 0, 'Yes': 1},
                           'pol_usage': {'Retired': 0, 'WorkPrivate': 1, 'Professional': 2, 'AllTrips': 3},
                           'drv_drv2': {'No': 0, 'Yes': 1},
                           'drv_sex1': {'F': 0, 'M': 1},
                           'drv_sex2': {'F': -1, 'M': 1, None: 0},
                           'vh_type': {'Tourism': 0, 'Commercial': 1, }
                           }

        def car_make_categories(car_make):
            if car_make in ['RENAULT', 'PEUGEOT', 'CITROEN', 'VOLKSWAGEN', 'FORD', 'MERCEDES BENZ']:
                return car_make
            else:
                return 'OTHER'

        def missing_geo_data(x):
            if x:
                return 1
            else:
                return 0

        def zero_vh_weight(weight, avg_weight):
            if weight < 100:
                return avg_weight
            else:
                return weight

        x_data_f = x_data_un.replace(cat_to_int_dict, inplace=False)

        x_data_f.vh_make = x_data_f['vh_make'].apply(lambda x: car_make_categories(x))

        ##MAKE INDEP
        avg_vh_weight = x_data_f['vh_weight'].mean()
        x_data_f.vh_weight = x_data_f['vh_weight'].apply(lambda x: zero_vh_weight(x, avg_vh_weight))

        vh_make_cols = pd.get_dummies(x_data_f.vh_make)
        vh_fuel_cols = pd.get_dummies(x_data_f.vh_fuel)
        city_dist_cols = pd.get_dummies(x_data_f.city_district_code)
        x_data_f = x_data_f.drop(['vh_fuel', 'vh_make', 'city_district_code'], axis=1)

        x_data_f['geoNA'] = (x_data_un['population'].isnull()).apply(lambda x: missing_geo_data(x))

        x_data_f = pd.concat([x_data_f, vh_make_cols, vh_fuel_cols, city_dist_cols], axis=1, sort=False)

        #NEED TO CHANGE MEANS SO THAT ITS INDEPENDENT
        means = x_data_f.mean()
        x_data_f = x_data_f.fillna(means)

        #SIMILIAARLY I NEED TO CHANGE THIS SCALER SO THAT ITS INDEPENDENT
        scaler = preprocessing.MinMaxScaler()
        if training == True:
            self.scaler = preprocessing.StandardScaler().fit(X_train)

        X = pd.DataFrame(scaler.fit_transform(x_data_f), columns=x_data_f.columns, index=x_data_f.index)

        return X